In [1]:
with open('token_vk.txt', 'r') as file_object:
    token_vk = file_object.read().strip()

with open('token_yandex.txt', 'r') as file_object:
    token_yandex = file_object.read().strip()
    
version = '5.126'

In [2]:
import requests
import os
import time
import tqdm
import tqdm.notebook
import json



class VkUser:
    url = 'https://api.vk.com/method/'

    def __init__(self, token_vk, version, user_id=None):
        self.user_id = user_id
        self.token_vk = token_vk
        self.version = version
        self.params = {
            'user_id': self.user_id,
            'access_token': self.token_vk,
            'v': self.version     
        }
        self.owner_id = requests.get(self.url + 'users.get', self.params).json()['response'][0]['id']
        if self.user_id is None:
            self.user_id = self.owner_id
        
        
        
        
    def get_photos(self):
        
        photos_url = self.url + 'photos.get'
        photos_params = {
            'owner_id': self.user_id,
            'album_id': 'profile',
            'extended': 1

        }
        res = requests.get(photos_url, params = {**self.params, **photos_params})
          

        res = res.json()['response']['items']
        
        likes = []        
        urllist = []
        photos = []
        
        for photo in res:
            urls = photo['sizes'][-1]['url']
            urllist.append([urls])
            file_name = photo['likes']['count']
            likes.append(file_name)
            file_size =  photo['sizes'][-1]['type']
            file_date = photo['date']
            if likes.count(file_name) != 1:
                file_name = str(file_name) +'_'+ str(file_date) + '.jpg'
            else:
                file_name = str(file_name) + '.jpg'
            photos.append({'file_name': file_name, 'size': file_size})
        
        
        return urllist, photos
    
    
            
    

In [3]:
class YaUploader:
    def __init__(self, token_yandex: str):
        self.token_yandex = token_yandex

    def upload(self, folder_name, file_name, file):
        """Метод загружает файл file на яндекс диск"""
        TOKEN = token_yandex
        HEADERS = {
            'Authorization': f'OAuth {TOKEN}'
        }
        
        
        response = requests.get(
            'https://cloud-api.yandex.net/v1/disk/resources/upload',
            params={
                'path': str(folder_name + '/' + file_name),
                'overwrite': True
            },
            headers=HEADERS
        )

        response.raise_for_status()

        href = response.json()['href']
        

        upload_response = requests.put(href, files={'file': file})
        upload_response.raise_for_status()
        print(f'Файл {file_name} успешно загружен!')
                
        
    
    def create_folder(self, folder_name):
        """Метод создает папку folder_name на яндекс диск"""
        
        TOKEN = token_yandex
        HEADERS = {
            'Authorization': f'OAuth {TOKEN}'
        }
        
        response = requests.put(
            'https://cloud-api.yandex.net/v1/disk/resources',
            params={
                'path': str(folder_name)
                
            },
             headers=HEADERS
        )
        
        response.raise_for_status()
        
        


In [4]:
""""Сохраняет фотографии на локальный диск в текущий каталог в папку photos"""
def save_photos(photos):
          
        photos_urls = photos[0]
        file_names = [file['file_name'] for file in photos[1]]
        print(file_names)
        
        i = 0
        for urls in photos_urls:
            print(urls)
            file_response = requests.get(urls[0])
            file_name = file_names[i]
            print(file_name)
            with open('photos/%s' % file_name, 'wb') as file:
                file.write(file_response.content)
            i += 1

In [5]:
""""Загружает файлы на Yandex Disk, по умолчанию папка для загрузки - vk_photos, кол-во фотографий = 5"""

def upload_to_yandex(photos, folder_name='vk_photos', photos_number=5):
        
        photos_urls = photos[0]
        file_names = [file['file_name'] for file in photos[1]] 
        
        if photos_number == '':
            photos_number = 5
        elif photos_number > len(photos_urls):
            print(f'Максимальное количество файлов для загрузки {len(photos_urls)}')
            photos_number = len(photos_urls)
        elif photos_number == 0:
            print('Выбрано 0 файлов для загрузки')
            return
        
        photos_urls = photos_urls[0:photos_number]
            
        uploader = YaUploader(token_yandex) 
        
        if folder_name == '':
            folder_name = 'vk_photos'
            
        uploader.create_folder(folder_name)
        
        i = 0
        
        pbar = tqdm.notebook.tqdm(photos_urls, desc='Uploading files')
            
        for urls in pbar:
            file_response = requests.get(urls[0])
            file_name = file_names[i]
            result = uploader.upload(folder_name, file_name, file_response.content)
            i += 1
            time.sleep(0.1)




In [6]:
def main():
    user_input = input('Введите id пользователя: ')
    if user_input == '':
        user_id = None
    else:
        user_id = user_input
        
    user_input = input('Введите Yandex токен: ')
    #     token_yandex = user_input
    user_input = input('Введите имя папки: ')
    folder_name = user_input
    user_input = input('Сколько фотографий сохранить на диск: ')
    photos_number = int(user_input)
    
        
    with open('token_vk.txt', 'r') as file_object:
        token_vk = file_object.read().strip()

    version = '5.126'
        
    user = VkUser(token_vk, version, user_id)
    res = user.get_photos()
    
    i = 0
    while(i == 0):
        i = 1
        try:
         upload_to_yandex(res, folder_name, photos_number)
        except requests.HTTPError:
            print('Папка с таким имеменем уже существует')
            i = 0
            user_input = input('Введите имя папки: ')
            folder_name = user_input
    
    
    with open('photos.json', 'w') as f:
        json.dump(res[1], f)
        print('Json файл сохранен на диск!')
    
    
    

In [7]:
main()

Введите id пользователя: 
Введите Yandex токен: 
Введите имя папки: new
Сколько фотографий сохранить на диск: 4


Файл 12.jpg успешно загружен!
Файл 3.jpg успешно загружен!
Файл 1.jpg успешно загружен!
Файл 1_1594036429.jpg успешно загружен!

Json файл сохранен на диск!
